# Lab 02: Regularized Regression Analysis
## Comparing Lasso, Ridge, and Elastic Net for Job Satisfaction Prediction

**Author:** Florencekumari Makwana  
**Date:** February 2026  

---

## Objective

Compare three regularization techniques (Lasso, Ridge, and Elastic Net) for predicting job satisfaction using employee demographic and work-related features.

## 1. Setup: Import Libraries

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully")

## 2. Data Loading & Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('../data/job_satisfaction_data.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Dataset information
print("Dataset Info:")
df.info()

print("\n" + "="*60)
print("Basic Statistics:")
print("="*60)
df.describe()

### Data Quality Check

**Important Finding:** One observation has `Years_of_Experience = -1`, which is a data anomaly. Given the small dataset (n=100), we retain this value for analysis, but in production this should be investigated and corrected.

In [ ]:
# Check for anomalies
print("Checking for negative values...")
print(f"\nMin Years_of_Experience: {df['Years_of_Experience'].min()}")
print(f"Record with negative experience:")
df[df['Years_of_Experience'] < 0]

### Exploratory Visualizations

In [ ]:
# Distribution of target variable
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['Job_Satisfaction'], bins=15, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Job Satisfaction Score', fontweight='bold')
axes[0].set_ylabel('Frequency', fontweight='bold')
axes[0].set_title('Distribution of Job Satisfaction', fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Box plot
axes[1].boxplot(df['Job_Satisfaction'], vert=True)
axes[1].set_ylabel('Job Satisfaction Score', fontweight='bold')
axes[1].set_title('Box Plot of Job Satisfaction', fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean: {df['Job_Satisfaction'].mean():.2f}")
print(f"Median: {df['Job_Satisfaction'].median():.2f}")
print(f"Std Dev: {df['Job_Satisfaction'].std():.2f}")

In [ ]:
# Correlation heatmap
numerical_cols = ['Age', 'Years_of_Experience', 'Hours_Worked_Per_Week', 'Salary', 'Job_Satisfaction']
correlation_matrix = df[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            fmt='.2f', square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numerical Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. Data Preprocessing

In [ ]:
# One-hot encoding for categorical variables
df_encoded = pd.get_dummies(df, columns=['Gender', 'Education_Level'], drop_first=True)

print("Encoded features:")
print(df_encoded.columns.tolist())
print(f"\nTotal features after encoding: {len(df_encoded.columns) - 1}")  # -1 for target

In [ ]:
# Separate features and target
X = df_encoded.drop(columns=['Job_Satisfaction'])
y = df_encoded['Job_Satisfaction']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

In [ ]:
# Split data: 80% train, 10% validation, 10% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print("Data Split Summary:")
print(f"Training:   {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"Validation: {X_val.shape[0]} samples ({X_val.shape[0]/len(X)*100:.0f}%)")
print(f"Test:       {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.0f}%)")

In [ ]:
# Feature standardization (CRITICAL for regularized regression)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print("✓ Features standardized (mean=0, std=1)")
print("\nScaled Training Data Sample:")
X_train_scaled.head()

## 4. Hyperparameter Tuning on Validation Set

In [ ]:
# Define alpha values to test
alpha_values = [0.01, 0.1, 1, 10, 100]

# Store results
validation_results = []

# Loop through alpha values
for alpha in alpha_values:
    print(f"\nTraining with alpha = {alpha}...")
    
    # Initialize models
    lasso_model = Lasso(alpha=alpha, random_state=42, max_iter=10000)
    ridge_model = Ridge(alpha=alpha, random_state=42)
    elastic_net_model = ElasticNet(alpha=alpha, random_state=42, max_iter=10000)
    
    # Train models
    lasso_model.fit(X_train_scaled, y_train)
    ridge_model.fit(X_train_scaled, y_train)
    elastic_net_model.fit(X_train_scaled, y_train)
    
    # Predictions on validation set
    y_pred_lasso = lasso_model.predict(X_val_scaled)
    y_pred_ridge = ridge_model.predict(X_val_scaled)
    y_pred_elastic_net = elastic_net_model.predict(X_val_scaled)
    
    # Compute metrics for each model
    for model_name, y_pred in [
        ('Lasso', y_pred_lasso),
        ('Ridge', y_pred_ridge),
        ('ElasticNet', y_pred_elastic_net)
    ]:
        validation_results.append({
            'Alpha': alpha,
            'Model': model_name,
            'MSE': mean_squared_error(y_val, y_pred),
            'MAE': mean_absolute_error(y_val, y_pred),
            'R²': r2_score(y_val, y_pred)
        })

# Create DataFrame
validation_df = pd.DataFrame(validation_results)

print("\n" + "="*70)
print("VALIDATION RESULTS")
print("="*70)
validation_df

In [ ]:
# Visualize performance vs alpha
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['MSE', 'MAE', 'R²']
colors = {'Lasso': '#FF6B6B', 'Ridge': '#4ECDC4', 'ElasticNet': '#95E1D3'}

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    for model in ['Lasso', 'Ridge', 'ElasticNet']:
        model_data = validation_df[validation_df['Model'] == model]
        ax.plot(model_data['Alpha'], model_data[metric], 
               marker='o', label=model, linewidth=2, color=colors[model])
    
    ax.set_xscale('log')
    ax.set_xlabel('Alpha (log scale)', fontsize=12, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} vs Alpha on Validation Set', fontsize=13, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    if metric == 'R²':
        ax.axhline(y=0, color='r', linestyle='--', linewidth=1, alpha=0.5)

plt.tight_layout()
plt.show()

### Find Best Alpha for Each Model

In [ ]:
# Find best alpha based on lowest MSE
best_models = {}

print("="*70)
print("BEST ALPHA VALUES (Based on Validation MSE)")
print("="*70)

for model in ['Lasso', 'Ridge', 'ElasticNet']:
    model_data = validation_df[validation_df['Model'] == model]
    best_row = model_data.loc[model_data['MSE'].idxmin()]
    best_models[model] = best_row
    
    print(f"\n{model}:")
    print(f"  Best Alpha: {best_row['Alpha']}")
    print(f"  MSE: {best_row['MSE']:.4f}")
    print(f"  MAE: {best_row['MAE']:.4f}")
    print(f"  R²: {best_row['R²']:.4f}")

# Store best alphas
best_alpha_lasso = best_models['Lasso']['Alpha']
best_alpha_ridge = best_models['Ridge']['Alpha']
best_alpha_elastic_net = best_models['ElasticNet']['Alpha']

## 5. Final Model Training & Test Set Evaluation

In [ ]:
# Train final models with best alpha values
final_lasso = Lasso(alpha=best_alpha_lasso, random_state=42, max_iter=10000)
final_ridge = Ridge(alpha=best_alpha_ridge, random_state=42)
final_elastic_net = ElasticNet(alpha=best_alpha_elastic_net, random_state=42, max_iter=10000)

# Train on full training set
final_lasso.fit(X_train_scaled, y_train)
final_ridge.fit(X_train_scaled, y_train)
final_elastic_net.fit(X_train_scaled, y_train)

print("✓ Final models trained")

In [ ]:
# Predictions on test set
y_pred_lasso_test = final_lasso.predict(X_test_scaled)
y_pred_ridge_test = final_ridge.predict(X_test_scaled)
y_pred_elastic_net_test = final_elastic_net.predict(X_test_scaled)

# Calculate metrics
test_results = pd.DataFrame({
    'Model': ['Lasso', 'Ridge', 'ElasticNet'],
    'Alpha': [best_alpha_lasso, best_alpha_ridge, best_alpha_elastic_net],
    'MSE': [
        mean_squared_error(y_test, y_pred_lasso_test),
        mean_squared_error(y_test, y_pred_ridge_test),
        mean_squared_error(y_test, y_pred_elastic_net_test)
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(y_test, y_pred_lasso_test)),
        np.sqrt(mean_squared_error(y_test, y_pred_ridge_test)),
        np.sqrt(mean_squared_error(y_test, y_pred_elastic_net_test))
    ],
    'MAE': [
        mean_absolute_error(y_test, y_pred_lasso_test),
        mean_absolute_error(y_test, y_pred_ridge_test),
        mean_absolute_error(y_test, y_pred_elastic_net_test)
    ],
    'R²': [
        r2_score(y_test, y_pred_lasso_test),
        r2_score(y_test, y_pred_ridge_test),
        r2_score(y_test, y_pred_elastic_net_test)
    ]
})

print("\n" + "="*70)
print("FINAL TEST SET RESULTS")
print("="*70)
test_results

### Key Finding:

**Lasso Regression achieved the best test performance!**
- R² = 0.704 (explains 70% of variance)
- MSE = 1.067 (lowest error)
- MAE = 0.874

Note: Rankings differ from validation (Ridge was best) vs test (Lasso is best), highlighting the importance of held-out test evaluation.

In [ ]:
# Visualize test performance
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

metrics = ['MSE', 'RMSE', 'MAE', 'R²']
colors = ['#FF6B6B', '#4ECDC4', '#95E1D3']

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    bars = ax.bar(test_results['Model'], test_results[metric], 
                 color=colors, alpha=0.8, edgecolor='black')
    
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} on Test Set', fontsize=13, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.3f}',
               ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.suptitle('Model Performance Comparison (Test Set)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Feature Importance Analysis

In [ ]:
# Compare coefficients
coefficients_df = pd.DataFrame({
    'Feature': X.columns,
    'Lasso': final_lasso.coef_,
    'Ridge': final_ridge.coef_,
    'ElasticNet': final_elastic_net.coef_
})

# Sort by absolute Ridge coefficient
coefficients_df['abs_ridge'] = coefficients_df['Ridge'].abs()
coefficients_df = coefficients_df.sort_values('abs_ridge', ascending=False)
coefficients_df = coefficients_df.drop('abs_ridge', axis=1)

print("\n" + "="*70)
print("MODEL COEFFICIENTS (Sorted by Ridge importance)")
print("="*70)
coefficients_df

In [ ]:
# Visualize coefficients
fig, ax = plt.subplots(figsize=(12, 8))

x_pos = np.arange(len(coefficients_df))
width = 0.25

ax.barh(x_pos - width, coefficients_df['Lasso'], width, 
       label='Lasso', alpha=0.8, color='#FF6B6B')
ax.barh(x_pos, coefficients_df['Ridge'], width, 
       label='Ridge', alpha=0.8, color='#4ECDC4')
ax.barh(x_pos + width, coefficients_df['ElasticNet'], width, 
       label='ElasticNet', alpha=0.8, color='#95E1D3')

ax.set_yticks(x_pos)
ax.set_yticklabels(coefficients_df['Feature'])
ax.set_xlabel('Coefficient Value (Standardized)', fontsize=12, fontweight='bold')
ax.set_title('Feature Importance Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### Key Insights:

1. **Years of Experience** is the dominant predictor (coefficient ≈ 1.44)
2. **Age** shows negative relationship (coefficient ≈ -0.65)
3. **Lasso feature selection**: Eliminated Salary, Hours_Worked_Per_Week, and Education_Level_High_School (coefficients = 0)
4. **Ridge retention**: Keeps all features with small non-zero coefficients

## 7. Predictions vs Actual

In [ ]:
# Scatter plots: Predicted vs Actual
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models_data = [
    ('Lasso', y_pred_lasso_test, best_alpha_lasso),
    ('Ridge', y_pred_ridge_test, best_alpha_ridge),
    ('ElasticNet', y_pred_elastic_net_test, best_alpha_elastic_net)
]

colors = {'Lasso': '#FF6B6B', 'Ridge': '#4ECDC4', 'ElasticNet': '#95E1D3'}

for idx, (model_name, y_pred, alpha) in enumerate(models_data):
    ax = axes[idx]
    
    # Scatter
    ax.scatter(y_test, y_pred, alpha=0.6, s=100, 
              edgecolors='black', linewidth=0.5, color=colors[model_name])
    
    # Perfect prediction line
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 
           'r--', linewidth=2, label='Perfect Prediction')
    
    ax.set_xlabel('Actual Job Satisfaction', fontsize=11, fontweight='bold')
    ax.set_ylabel('Predicted Job Satisfaction', fontsize=11, fontweight='bold')
    ax.set_title(f'{model_name} (α = {alpha})', fontsize=13, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # R² score
    r2 = r2_score(y_test, y_pred)
    ax.text(0.05, 0.95, f'R² = {r2:.3f}', transform=ax.transAxes,
           fontsize=11, verticalalignment='top', fontweight='bold',
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('Predicted vs Actual Job Satisfaction (Test Set)', 
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 8. Residual Analysis

In [ ]:
# Residual plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (model_name, y_pred, alpha) in enumerate(models_data):
    ax = axes[idx]
    residuals = y_test - y_pred
    
    ax.scatter(y_pred, residuals, alpha=0.6, s=100,
              edgecolors='black', linewidth=0.5, color=colors[model_name])
    ax.axhline(y=0, color='red', linestyle='--', linewidth=2)
    
    ax.set_xlabel('Predicted Job Satisfaction', fontsize=11, fontweight='bold')
    ax.set_ylabel('Residuals', fontsize=11, fontweight='bold')
    ax.set_title(f'{model_name} Residual Plot', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('Residual Analysis (Test Set)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 9. Conclusions

### Best Model: **Lasso Regression (α = 0.1)**

**Performance:**
- R² = 0.704 (explains 70% of job satisfaction variance)
- MSE = 1.067
- MAE = 0.874

**Key Findings:**

1. **Years of Experience** is the strongest predictor (coefficient ≈ 1.44)
2. **Age** shows negative relationship when controlling for experience (coefficient ≈ -0.65)
3. **Salary** and **Hours Worked** had minimal impact (eliminated by Lasso)
4. Low to moderate regularization (α = 0.1-1.0) optimal for this dataset
5. Model rankings differ between validation and test sets, emphasizing importance of held-out evaluation

**Limitations:**
- Small sample size (n=100) and tiny test/validation sets (n=10 each)
- Data quality issue: Years_of_Experience = -1
- Limited hyperparameter search (only 5 alpha values)

**Recommendations:**
1. Collect more data (target n ≥ 500)
2. Implement k-fold cross-validation
3. Clean data anomalies
4. Test interaction terms and polynomial features
5. Compare with non-linear models (Random Forest, XGBoost)

---

**Analysis Complete!** 🎉